# Интеграция данных
***

## Имеющиеся данные

- [спутниковые снимки Sentinel](../../notebooks/1.1-data-review-sentinel.ipynb);
- [климатические данные](../../notebooks/1.3-data-review-wrf-hrrr.ipynb);
- [данные о влажности почвы](../../notebooks/1.4-data-review-moisture.ipynb);
- [исторические данные об урожайности](../../notebooks/1.2-data-review-usda.ipynb).

Необходимо их объединить в единое целое.
***

## Ограничение накладываемые данными

- **Sentinel:** Снимки одной локации предоставлены за 1 и 15 числа каждого месяца.

- **USDA:** Данные об урожайности предоставлены для округов (counties).

Единый датасет должен быть структурирован так, чтобы соответствовать ограничениям.
***

## Структура будущего датасета

```plaintext
interim
    |
    ├── data.csv                                            <- Табличные данные
    |
    |
    ├── <fips>-<year>-<month>-<day>                         <- Набор снимков для одного округа и даты
    |   |
    |   ├── 1.npy
    |   |
    |   ├── 2.npy
    |   |
    |   └── ...
    |
    └── ...                                                 <- Остальные наборы наборы снимков
```
***

## Интеграция данных

- Возьмем данные за *1 и 15 дни каждого месяца для каждого округа* и объединим их в одну таблицу.
- Удалим метаданные из таблиц.
- Усредним данные для каждого округа (fips).
- Удалим данные, для которых нет значения для целевой переменной (`YIELD, MEASURED IN BU / ACRE`) (для тех, которых нет данных в `USDA`).
- Добавим столбец `images` - путь к набору снимков.
- Удалим данные, для которых нет снимков.
- Разделим признаки и таргет. `PRODUCTION, MEASURED IN BU` можно удалить.
- Добавим столбец `target_year`, который ставит в соответствие объект и таргет. В частности, для периода месяцев *ноябрь-декабрь* `target_year = year + 1`, т.к. сбор урожая кукурузы происходит в *сентябре-октябре*.

In [1]:
import os
import gc
import h5py
import numpy as np
import pandas as pd
from pathlib import Path

PATH_DATA = Path("../../data/raw")
PATH_PROCESSED = Path("../../data/interim")
PATH_SENTINEL = PATH_DATA / "Sentinel"
PATH_HRRR = PATH_DATA / "WRF-HRRR"
PATH_ERA5 = PATH_DATA / "ERA5-Land-Moisture"
PATH_USDA = PATH_DATA / "USDA"

In [2]:
def get_all_files(path: Path) -> list[Path]:
    """Возвращает все пути к файлам данных из подкаталогов

    Args:
        path (Path): путь к каталогу данных

    Returns:
        list[Path]: список путей к файлам
    """
    return [
        file
        for dir_ in path.iterdir()
        for subdir in dir_.iterdir()
        for file in subdir.iterdir()
    ]

In [3]:
files_sentinel = get_all_files(PATH_SENTINEL)
files_hrrr = get_all_files(PATH_HRRR)
files_era5 = get_all_files(PATH_ERA5)
files_usda = get_all_files(PATH_USDA)

In [4]:
def read_csv_files(files: list[Path]) -> pd.DataFrame:
    """Читает файлы .csv одной структуры из списка файлов files
    и возвращает их как pd.DataFrame

    Args:
        files (list[Path]): список файлов

    Returns:
        pd.DataFrame: DataFrame
    """
    return pd.concat(
        [pd.read_csv(file) for file in files], axis=0, ignore_index=True
    )

In [5]:
df_hrrr = read_csv_files(files_hrrr)
df_era5 = read_csv_files(files_era5)
df_usda = read_csv_files(files_usda)

In [6]:
print(f"df_hrrr columns:\n{df_hrrr.columns.to_list()}\n")
print(f"df_era5 columns:\n{df_era5.columns.to_list()}\n")
print(f"df_usda columns:\n{df_usda.columns.to_list()}\n")

df_hrrr columns:
['Year', 'Month', 'Day', 'Daily/Monthly', 'State', 'County', 'FIPS Code', 'Grid Index', 'Lat (llcrnr)', 'Lon (llcrnr)', 'Lat (urcrnr)', 'Lon (urcrnr)', 'Avg Temperature (K)', 'Max Temperature (K)', 'Min Temperature (K)', 'Precipitation (kg m**-2)', 'Relative Humidity (%)', 'Wind Gust (m s**-1)', 'Wind Speed (m s**-1)', 'U Component of Wind (m s**-1)', 'V Component of Wind (m s**-1)', 'Downward Shortwave Radiation Flux (W m**-2)', 'Vapor Pressure Deficit (kPa)']

df_era5 columns:
['year', 'month', 'day', 'hour', 'fips', 'state', 'latitude', 'longitude', 'src', 'swvl1', 'swvl2', 'swvl3']

df_usda columns:
['commodity_desc', 'reference_period_desc', 'year', 'state_ansi', 'state_name', 'county_ansi', 'county_name', 'asd_code', 'asd_desc', 'domain_desc', 'source_desc', 'agg_level_desc', 'PRODUCTION, MEASURED IN BU', 'YIELD, MEASURED IN BU / ACRE']



In [7]:
def get_fips(df_usda: pd.DataFrame) -> pd.Series:
    """Получает fips используя поля state_ansi и county_ansi

    Args:
        df_usda (pd.DataFrame): USDA dataset

    Returns:
        pd.Series: fips codes
    """
    states_fips = df_usda["state_ansi"].astype(str).str.zfill(2)
    counties_fips = df_usda["county_ansi"].astype(str).str.zfill(3)
    fips = states_fips + counties_fips
    return fips

In [8]:
df_usda["fips"] = get_fips(df_usda).astype(np.int64)  #

In [9]:
df_hrrr.rename(
    {"Year": "year", "Month": "month", "Day": "day", "FIPS Code": "fips"},
    axis=1,
    inplace=True,
)


In [10]:
df_hrrr.query("(day == 1) | (day == 15)", inplace=True)
df_era5.query("(day == 1) | (day == 15)", inplace=True)
df_usda.query(
    "(state_name == 'ILLINOIS') | (state_name == 'IOWA')", inplace=True
)

In [11]:
# see ../../notebooks/1.3-data-review-wrf-hrrr.ipynb
df_hrrr.dropna(axis=0, inplace=True)
df_hrrr.drop(
    [
        "State",
        "County",
        "Grid Index",
    ],
    axis=1,
    inplace=True,
)

In [12]:
agg_dict = {
    column: "mean"
    for column in [
        "Lat (llcrnr)",
        "Lon (llcrnr)",
        "Lat (urcrnr)",
        "Lon (urcrnr)",
        "Avg Temperature (K)",
        "Max Temperature (K)",
        "Min Temperature (K)",
        "Precipitation (kg m**-2)",
        "Relative Humidity (%)",
        "Wind Gust (m s**-1)",
        "Wind Speed (m s**-1)",
        "U Component of Wind (m s**-1)",
        "V Component of Wind (m s**-1)",
        "Downward Shortwave Radiation Flux (W m**-2)",
        "Vapor Pressure Deficit (kPa)",
    ]
}
agg_dict["Lat (llcrnr)"] = "min"
agg_dict["Lon (llcrnr)"] = "min"
agg_dict["Lat (urcrnr)"] = "max"
agg_dict["Lon (urcrnr)"] = "max"
df_hrrr = (
    df_hrrr.groupby(["fips", "year", "month", "day"])
    .agg(agg_dict)
    .reset_index()
)

In [13]:
df_era5.drop(["hour", "state", "latitude", "longitude"], axis=1, inplace=True)
df_era5 = (
    df_era5.groupby(["fips", "year", "month", "day"]).mean().reset_index()
)

In [14]:
df_usda = df_usda[df_usda["commodity_desc"] == "CORN"]
df_usda.drop(
    [
        "commodity_desc",
        "reference_period_desc",
        "state_ansi",
        "state_name",
        "county_ansi",
        "county_name",
        "asd_code",
        "asd_desc",
        "domain_desc",
        "source_desc",
        "agg_level_desc",
    ],
    axis=1,
    inplace=True,
)

In [15]:
data = pd.merge(df_usda, df_hrrr, how="left", on=["year", "fips"])
del df_usda, df_hrrr
gc.collect()

data = pd.merge(data, df_era5, how="left", on=["year", "month", "day", "fips"])
del df_era5
gc.collect();

In [16]:
display(data.shape)
display(data.columns)

(26016, 25)

Index(['year', 'PRODUCTION, MEASURED IN BU', 'YIELD, MEASURED IN BU / ACRE',
       'fips', 'month', 'day', 'Lat (llcrnr)', 'Lon (llcrnr)', 'Lat (urcrnr)',
       'Lon (urcrnr)', 'Avg Temperature (K)', 'Max Temperature (K)',
       'Min Temperature (K)', 'Precipitation (kg m**-2)',
       'Relative Humidity (%)', 'Wind Gust (m s**-1)', 'Wind Speed (m s**-1)',
       'U Component of Wind (m s**-1)', 'V Component of Wind (m s**-1)',
       'Downward Shortwave Radiation Flux (W m**-2)',
       'Vapor Pressure Deficit (kPa)', 'src', 'swvl1', 'swvl2', 'swvl3'],
      dtype='object')

In [17]:
# Новые имена столбцов
new_columns = {
    "year": "year",
    "month": "month",
    "day": "day",
    "fips": "fips",
    "Lat (llcrnr)": "lat_lower_left",
    "Lon (llcrnr)": "lon_lower_left",
    "Lat (urcrnr)": "lat_upper_right",
    "Lon (urcrnr)": "lon_upper_right",
    "Avg Temperature (K)": "temperature_avg",
    "Max Temperature (K)": "temperature_max",
    "Min Temperature (K)": "temperature_min",
    "Precipitation (kg m**-2)": "precipitation",
    "Relative Humidity (%)": "humidity_relative",
    "Wind Gust (m s**-1)": "wind_gust",
    "Wind Speed (m s**-1)": "wind_speed",
    "U Component of Wind (m s**-1)": "wind_u_component",
    "V Component of Wind (m s**-1)": "wind_v_component",
    "Downward Shortwave Radiation Flux (W m**-2)": "solar_radiation_downward",
    "Vapor Pressure Deficit (kPa)": "vapor_pressure_deficit",
    "src": "skin_reservoir_content",
    "swvl1": "soil_water_vol_layer1",
    "swvl2": "soil_water_vol_layer2",
    "swvl3": "soil_water_vol_layer3",
    "YIELD, MEASURED IN BU / ACRE": "yield_bu_per_acre",
    "PRODUCTION, MEASURED IN BU": "production_bu",
}

data = data[new_columns.keys()]
data.rename(new_columns, inplace=True, axis=1)

In [18]:
for to_int in ["year", "month", "day", "fips"]:
    data[to_int] = data[to_int].astype(np.int32)

In [19]:
data.dtypes

year                          int32
month                         int32
day                           int32
fips                          int32
lat_lower_left              float64
lon_lower_left              float64
lat_upper_right             float64
lon_upper_right             float64
temperature_avg             float64
temperature_max             float64
temperature_min             float64
precipitation               float64
humidity_relative           float64
wind_gust                   float64
wind_speed                  float64
wind_u_component            float64
wind_v_component            float64
solar_radiation_downward    float64
vapor_pressure_deficit      float64
skin_reservoir_content      float64
soil_water_vol_layer1       float64
soil_water_vol_layer2       float64
soil_water_vol_layer3       float64
yield_bu_per_acre           float64
production_bu               float64
dtype: object

In [20]:
data["images"] = (
    data["fips"].astype(str)
    + "-"
    + data["year"].astype(str)
    + "-"
    + data["month"].astype(str).str.zfill(2)
    + "-"
    + data["day"].astype(str).str.zfill(2)
)

In [21]:
data["images"]

0        17001-2020-01-01
1        17001-2020-01-15
2        17001-2020-02-01
3        17001-2020-02-15
4        17001-2020-03-01
               ...       
26011    19197-2019-10-15
26012    19197-2019-11-01
26013    19197-2019-11-15
26014    19197-2019-12-01
26015    19197-2019-12-15
Name: images, Length: 26016, dtype: object

Копируем изображения. Если для набора изображений нет данных в таблице `data`, то не копируем

In [22]:
if not PATH_PROCESSED.exists():
    os.mkdir(PATH_PROCESSED)
for file in files_sentinel:
    with h5py.File(file, "r") as h5:
        for fips, attrs0 in h5.items():
            for date, attrs1 in attrs0.items():
                dir_name = f"{fips}-{date}"
                if dir_name not in data["images"].values:
                    continue

                new_dir = PATH_PROCESSED / dir_name
                if new_dir.exists():
                    continue

                os.mkdir(new_dir)
                for i, image in enumerate(attrs1["data"][:]):
                    np.save(new_dir / f"{i}.npy", image)

Удаляем те объекты из `data`, для которых нет снимков

In [23]:
images = pd.Series(
    [str(path.name) for path in PATH_PROCESSED.iterdir() if not path.is_file()]
)

In [24]:
data = data[data["images"].isin(images)]

Добавим столбец `target_year`

In [25]:
data["target_year"] = np.where(
    data["month"] >= 11, data["year"] + 1, data["year"]
)

На всякий отсортируем и потом разделим признаки и таргет

In [26]:
data.sort_values(by=["year", "month", "day", "fips"], inplace=True)

In [27]:
X = data.drop(["production_bu", "yield_bu_per_acre"], axis=1)
y = data[["year", "fips", "yield_bu_per_acre"]].drop_duplicates()

Сохраняем

In [28]:
X.to_csv(PATH_PROCESSED / "X.csv", index=False)
y.to_csv(PATH_PROCESSED / "y.csv", index=False)

In [31]:
display(X.shape)
display(y.shape)

(23851, 25)

(995, 3)